In [1]:
import pandas as pd

In [188]:
data = pd.read_csv('horse_data.csv', na_values = '?').iloc[:,[1, 3, 4, 10, 12, 17, 19, 26]]
data.columns = [
    'age',
    'rectal temperature',
    'pulse',
    'pain',
    'abdominal distension',
    'abdomen',
    'total protein',
    'type of surgical lesion'
]
data

,age,rectal temperature,pulse,pain,abdominal distension,abdomen,total protein,type of surgical lesion
0,1,39.2,88.0,3.0,2.0,2.0,85.0,0
1,1,38.3,40.0,3.0,1.0,1.0,6.7,0
2,9,39.1,164.0,2.0,4.0,NaN,7.2,0
3,1,37.3,104.0,NaN,NaN,NaN,7.4,0
4,1,NaN,NaN,2.0,2.0,3.0,NaN,0
...,...,...,...,...,...,...,...,...
294,1,NaN,120.0,2.0,NaN,5.0,65.0,0
295,1,37.2,72.0,4.0,3.0,4.0,NaN,0
296,1,37.5,72.0,4.0,3.0,5.0,6.8,0
297,1,36.5,100.0,3.0,3.0,4.0,6.0,0


In [189]:
data.max()

age                           9.0
rectal temperature           40.8
pulse                       184.0
pain                          5.0
abdominal distension          4.0
abdomen                       5.0
total protein                89.0
type of surgical lesion    2209.0
dtype: float64

In [190]:
data.min()

age                         1.0
rectal temperature         35.4
pulse                      30.0
pain                        1.0
abdominal distension        1.0
abdomen                     1.0
total protein               3.3
type of surgical lesion     0.0
dtype: float64

In [191]:
data.mean().round()

age                         2.0
rectal temperature         38.0
pulse                      72.0
pain                        3.0
abdominal distension        2.0
abdomen                     4.0
total protein              25.0
type of surgical lesion     7.0
dtype: float64

In [192]:
data.median()

age                         1.0
rectal temperature         38.2
pulse                      64.0
pain                        3.0
abdominal distension        2.0
abdomen                     4.0
total protein               7.5
type of surgical lesion     0.0
dtype: float64

In [193]:
data.round().mode()

,age,rectal temperature,pulse,pain,abdominal distension,abdomen,total protein,type of surgical lesion
0,1,38.0,48.0,3.0,1.0,5.0,7.0,0


Вкратце, в среднем температура чуть выше средней (норма 37.8). Можно сказать, что большая часть пациентов все же имеет некоторые осложнения. Пульс повышенный (норма 30-40). Боль, в основном, легкая прерывистая. Вздутие живота не было, хотя у некоторых слабое все же есть. У большинства брюшная полость растянута в толстом кишечнике. Уровень белка у пациентов в норме (норма 6-7.5). До операций дело не дошло. 



Видно, что есть пациент, а может даже несколько, который явно старше остальных (9 месяцев), но это значение нельзя назвать выбросом, потому что похоже на естественные данные. Эти данные не игнорируем, их отсутствие даст неправильную статистику!

А вот с количеством белка точно что-то не так. Есть значение 89 - максимальное, и минимальное - 3. Скорее всего значения выше 15 будут выбросом, потому что такие невозможны. А около 3, я думаю, реальны, поэтому минимум за выброс не берем. Возможно, выбросы тут преднамеренные или из-за ошибки ввода данных. Чтобы сильно не искажать реальные данные, можно попробовать заменить выбросы на медиану.

Аналогично с типом операции (столбец 'type of surgical lesion') видно выбросы. Максимальное значение таблицы > 2000, а по условию - 11. Поэтому все, что выше 11, будем считать выбросом. Возможно, выбросы тут преднамеренные или из-за ошибки ввода данных. Т.к. значения дискретные, выбросы заменим модой.

In [194]:
# заменим выбросы на медиану в столбце total protein
data.loc[data['total protein'] > 15, 'total protein'] = data['total protein'].median()

In [195]:
# заменим выбросы на моду в столбце type of surgical lesion
data.loc[data['type of surgical lesion'] > 11, 'type of surgical lesion'] = data['type of surgical lesion'].mode()

In [196]:
# новые средние значения
data.mean().round()

age                         2.0
rectal temperature         38.0
pulse                      72.0
pain                        3.0
abdominal distension        2.0
abdomen                     4.0
total protein               7.0
type of surgical lesion     0.0
dtype: float64

In [197]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      299 non-null    int64  
 1   rectal temperature       239 non-null    float64
 2   pulse                    275 non-null    float64
 3   pain                     244 non-null    float64
 4   abdominal distension     243 non-null    float64
 5   abdomen                  181 non-null    float64
 6   total protein            266 non-null    float64
 7   type of surgical lesion  298 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.8 KB


In [198]:
# количество пропусков 
for col in data.columns:
    pct_missing = data[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

age - 0.0%
rectal temperature - 20.1%
pulse - 8.0%
pain - 18.4%
abdominal distension - 18.7%
abdomen - 39.5%
total protein - 11.0%
type of surgical lesion - 0.3%


Пропуски заменим медианным значением, чтобы не исказить данные

In [200]:
data['rectal temperature'].fillna(data['rectal temperature'].median(), inplace=True)
data['pulse'].fillna(data['pulse'].median(), inplace = True)
data['pain'].fillna(data['pain'].median(), inplace=True)
data['abdominal distension'].fillna(data['abdominal distension'].median(), inplace=True)
data['total protein'].fillna(data['total protein'].median(), inplace=True)
data['abdomen'].fillna(data['abdomen'].median(), inplace=True)
data['type of surgical lesion'].fillna(data['type of surgical lesion'].median(), inplace=True)

In [202]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      299 non-null    int64  
 1   rectal temperature       299 non-null    float64
 2   pulse                    299 non-null    float64
 3   pain                     299 non-null    float64
 4   abdominal distension     299 non-null    float64
 5   abdomen                  299 non-null    float64
 6   total protein            299 non-null    float64
 7   type of surgical lesion  299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.8 KB


In [203]:
data

,age,rectal temperature,pulse,pain,abdominal distension,abdomen,total protein,type of surgical lesion
0,1,39.2,88.0,3.0,2.0,2.0,7.5,0.0
1,1,38.3,40.0,3.0,1.0,1.0,6.7,0.0
2,9,39.1,164.0,2.0,4.0,4.0,7.2,0.0
3,1,37.3,104.0,3.0,2.0,4.0,7.4,0.0
4,1,38.2,64.0,2.0,2.0,3.0,7.5,0.0
...,...,...,...,...,...,...,...,...
294,1,38.2,120.0,2.0,2.0,5.0,7.5,0.0
295,1,37.2,72.0,4.0,3.0,4.0,7.5,0.0
296,1,37.5,72.0,4.0,3.0,5.0,6.8,0.0
297,1,36.5,100.0,3.0,3.0,4.0,6.0,0.0
